In [209]:
#imports
import os, shutil, fnmatch, time, tkinter
from tkinter.filedialog import askdirectory
from pathlib import Path

In [210]:
#prompt user for the file directory. Will open as a popup window named "tk"
tk_root = tkinter.Tk()
path_input = askdirectory(title='Select Folder')  # shows dialog box and return the path
print(path_input)
tk_root.destroy()

E:/Microscope Images/Code_processed_eclipse_data/20220404_NHDFneo_LY294_GDC_2/Sorted/tissue_ai


In [211]:
stage_pos_chars = []
for file in os.listdir(path_input):
    if file.endswith('.nd'):
        basename = file.split('.',1)[0]
        with open(path_input + '/' +file, 'r') as nd_file:
            for l_no, line in enumerate(nd_file):
                if '"NStagePositions"' in line:
                    stage_pos_nd = [int(s) for s in line.split() if s.isdigit()][-1]
                if '"NTimePoints"' in line:
                    timepoints_nd = [int(s) for s in line.split() if s.isdigit()][-1]
                if '"Stage' in line:
                    spc = line.split(', "')[-1]
                    stage_pos_chars.append(spc[0:len(spc)-2])


In [212]:
for file in os.listdir(path_input):
    if os.path.isdir(path_input + '/' + file):
        #rename file for correct sort syntax
                folder = file.split('s', 1)[-1]

                for num in range(2-len(folder)):
                    folder = '0'+folder
                newfile = 's'+folder
                os.rename(path_input + '/' + file, path_input + '/' + newfile)


In [213]:
import pandas as pd
list_pos= []
tlist = [T*0.5 for T in range(1,timepoints_nd+1)]
wa_df = pd.DataFrame({'Frame': range(1,timepoints_nd+1),
                      'Time':  tlist})
for file in os.listdir(path_input):
    if os.path.isdir(path_input+'/'+file):

        list_pos.append(file)
        wa_df[file] = pd.read_table(path_input+'/'+file+'/segment_ph1/wound_area_vs_frame.txt',header = None)

header1 = wa_df.columns
header2 = ['t','Hours']+ stage_pos_chars
wa_df.columns = [header1, header2]
display(wa_df)

xls_name = path_input.split('Code_processed_eclipse_data/',1)[-1].split('/',1)[0]
#wa_df.to_excel(r'E:/Microscope Images/Code_processed_eclipse_data/Wound_area_code_output.xlsx', sheet_name=xls_name+'_'+ basename)

def append_to_excel(fpath, df, sheet_name):
    with pd.ExcelWriter(fpath, mode="a") as f:
        df.to_excel(f, sheet_name=sheet_name)

append_to_excel('E:/Microscope Images/Code_processed_eclipse_data/Wound_area_code_output.xlsx', wa_df,xls_name)


,Frame,Time,s01,s02,s03,s04,s05,s06,s07,s08,...,s58,s59,s60,s61,s62,s63,s64,s65,s66,s67
,t,Hours,A1,A2,A4,A5,A7,A8,A9,A11,...,G5,G6,G7,G8,G9,G10,H8,H4,H2,H1
0,1,0.5,3940.0,2772.0,3112.0,4570.0,1506.0,722.0,3119.0,6036.0,...,2914.0,1955.0,1675.0,3413.0,4241.0,171.0,770.0,5431.0,2786.0,1173.0
1,2,1.0,4534.0,856.0,914.0,4595.0,1064.0,699.0,3746.0,6220.0,...,2511.0,1936.0,1688.0,3732.0,4746.0,163.0,2273.0,4894.0,3357.0,1898.0
2,3,1.5,2703.0,873.0,2383.0,5068.0,962.0,1407.0,4112.0,6921.0,...,1864.0,2792.0,2227.0,5142.0,6058.0,325.0,1545.0,7121.0,4106.0,2440.0
3,4,2.0,2930.0,802.0,930.0,5123.0,892.0,1528.0,4250.0,7027.0,...,2253.0,3413.0,2853.0,6122.0,6875.0,692.0,2129.0,8713.0,4823.0,2851.0
4,5,2.5,2978.0,839.0,2264.0,5204.0,833.0,1686.0,4224.0,7011.0,...,2625.0,3568.0,3292.0,6716.0,7365.0,994.0,1629.0,9712.0,5226.0,3112.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,74,37.0,2490.0,3467.0,878.0,5140.0,2750.0,5235.0,3292.0,7738.0,...,1488.0,121.0,103.0,73.0,332.0,570.0,99.0,77.0,367.0,337.0
74,75,37.5,2670.0,3394.0,961.0,4844.0,2632.0,5238.0,3195.0,7657.0,...,1544.0,330.0,120.0,73.0,214.0,515.0,177.0,490.0,442.0,326.0
75,76,38.0,2414.0,3509.0,967.0,4855.0,1634.0,5111.0,3169.0,7514.0,...,833.0,495.0,69.0,99.0,284.0,195.0,183.0,219.0,416.0,162.0


C:\Users\anish\anaconda3\envs\wound-compute-env\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
